## 각종 필요 패키지 import

In [63]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import datetime
from dateutil.relativedelta import relativedelta
from collections import defaultdict

## CSV 파일 line 수를 저장하는 클래스 정의

In [2]:
class DLQC():
    def __init__(self, years):
        for year in years:
            setattr(self, f'min_{year}', defaultdict(lambda: np.zeros(shape=12)))
            setattr(self, f'qcd_{year}', defaultdict(lambda: np.zeros(shape=12)))

# dlqc.min_2021[stn_id][0:12] = lines

## 이름 유효성 검사 및 csv line 세는 함수 정의

In [3]:
def validation_test_for_naming(m_q, stn_id, date):
    return m_q != 'min' or m_q != 'qcd' or stn_id >= 1000 or date % 100 > 12


def get_lines_with_csv_path(path):
    chunk = 1024*1024   # Process 1 MB at a time.
    f = np.memmap(path, mode='r')
    num_newlines = sum(np.sum(f[i:i+chunk] == ord('\n')) for i in range(0, len(f), chunk))
    del f
    return num_newlines

## EDA

먼저 모든 csv 파일을 읽고 line 수만 저장해놓는다.

In [4]:
data_dir = 'CSV3DB'
header_name = ['time', 'ta', 'wd', 'ws', 'rn', 'pa', 're', 'sd', 'hm']
years = os.listdir(data_dir)

dlqc = DLQC(years)

for year in years:
    pbar = tqdm(os.listdir(os.path.join('CSV3DB', year)), desc=year)
    for csv_name in pbar:
        if csv_name.endswith('csv'):
            _, _, m_q, stn_id, date = csv_name.split('_')
            stn_id = int(stn_id)
            date, _ = date.split('.')
            date = int(date)
            if validation_test_for_naming(m_q, stn_id, date):
                getattr(dlqc, f'{m_q}_{year}')[stn_id][date % 100 - 1] = get_lines_with_csv_path(os.path.join('CSV3DB', year, csv_name))
            else:
                raise Exception(f'{csv_name} is invalid name.')      
        else:
            raise Exception(f'{csv_name} is invalid name.')



2022: 100%|██████████| 16330/16330 [00:52<00:00, 313.00it/s]


In [5]:
# station id가 몇 개인지 확인
len(dlqc.min_2021), len(dlqc.qcd_2021), len(dlqc.min_2022), len(dlqc.qcd_2022)

(761, 761, 754, 753)

2021년과 2022년의 stn_id 개수가 서로 맞지 않는다.

또, 2022년의 min과 qcd의 stn_id 개수가 같지 않다.

먼저 2021년 min, qcd간의 stn_id와 2022년 min, qcd간의 stn_id가 서로 같은지 확인한다.

In [6]:
# 2021년은 같지만 2022년은 다르다.
set(dlqc.min_2021) == set(dlqc.qcd_2021), set(dlqc.min_2022) == set(dlqc.qcd_2022)

(True, False)

In [7]:
set(dlqc.min_2022) - set(dlqc.qcd_2022), set(dlqc.qcd_2022) - set(dlqc.min_2022)

({111}, set())

2022년 111번은 min은 있지만 qcd가 없다.

2022년 111번 min csv 이름 찾기

In [8]:
no_qcd_list = list(filter(lambda x: x.startswith('dlqc_aws3_min_111'), os.listdir(os.path.join(data_dir, '2022'))))
no_qcd_list

['dlqc_aws3_min_111_202207.csv']

In [9]:
# 하나의 csv파일이므로 열어서 확인
temp_df = pd.read_csv(os.path.join(data_dir, '2022', no_qcd_list[0]),
                      names=header_name)
temp_df.describe()

,ta,wd,ws,rn,pa,re,sd,hm
count,56.000000,56.000000,56.000000,56.0,56.0,56.0,56.0,56.000000
mean,30.235714,185.366071,1.091071,0.0,-999.0,-999.0,-999.0,-661.232143
std,1.508452,144.805072,0.524760,0.0,0.0,0.0,0.0,495.211286
min,26.600000,0.000000,0.100000,0.0,-999.0,-999.0,-999.0,-999.000000
25%,29.650000,39.550000,0.700000,0.0,-999.0,-999.0,-999.0,-999.000000
50%,30.500000,190.050000,1.100000,0.0,-999.0,-999.0,-999.0,-999.000000
75%,31.500000,326.125000,1.300000,0.0,-999.0,-999.0,-999.0,48.000000
max,32.800000,359.800000,2.400000,0.0,-999.0,-999.0,-999.0,58.000000


In [10]:
# 적어도 1시간의 timestep이 필요한데, 없으므로 버려도 될 것으로 판단, 초기화한다.
del(dlqc.min_2022[111])

In [11]:
set(dlqc.min_2021) == set(dlqc.qcd_2021), set(dlqc.min_2022) == set(dlqc.qcd_2022)

(True, True)

지금까지 2021년 내, 2022년 내에서 비교했다면, 이젠 2021년과 2022년 간의 stn_id를 비교한다.

In [30]:
# 2021년에만 있는 stn_id : {0, 89, 111, 197, 220, 484, 562, 684, 880, 894, 895, 981}
# 2022년에만 있는 stn_id : {34, 347, 348, 994}
set(dlqc.min_2021) - set(dlqc.min_2022), set(dlqc.min_2022) - set(dlqc.min_2021)

({0, 89, 111, 197, 220, 484, 562, 684, 880, 894, 895, 981},
 {34, 347, 348, 994})

결국 모든 csv파일을 pandas로 읽어 다음과 같은 내용을 확인해보려 한다.

stn별 어떤 기상 관측 요소를 사용하는지,

각 월별로 min과 qcd가 서로 일치하는 time step을 갖고있는지,

또 얼마나 많은 결측지가 있는지 등을 확인해본다.

## 각종 확인 함수 정의

In [73]:
def is_df_time_has_full_range(series, year, month):
    full_range = pd.date_range(datetime(int(year), month, 1), datetime(int(year), month, 1) + relativedelta(months=1), freq='T')
    return len(series) == len(full_range) and series == full_range


def is_dfs_index_has_same_range(series1, series2):
    return len(series1) == len(series2) and series1 == series2 

In [74]:
for year in years:
    pbar = tqdm(getattr(dlqc, f'min_{year}'), desc=year)
    for stn_id in pbar:
        min_list = getattr(dlqc, f'min_{year}')[stn_id] > 0
        qcd_list = getattr(dlqc, f'qcd_{year}')[stn_id] > 0
        for month_idx in np.where(min_list | qcd_list)[0]:
            # min, qcd 둘 다 있는 월
            if month_idx in np.where(min_list & qcd_list)[0]:
                min_df = pd.read_csv(os.path.join(data_dir, year, f'dlqc_aws3_min_{stn_id:03d}_{year}{month_idx+1:02d}.csv'),
                                     header=None, names=header_name, date_format='%Y.%m.%d %H:%M:%S', parse_dates=[0], index_col=[0])
                qcd_df = pd.read_csv(os.path.join(data_dir, year, f'dlqc_aws3_qcd_{stn_id:03d}_{year}{month_idx+1:02d}.csv'),
                                     header=None, names=header_name, date_format='%Y.%m.%d %H:%M:%S', parse_dates=[0], index_col=[0])
                
                if is_df_time_has_full_range(min_df.index, year, month_idx + 1) and is_df_time_has_full_range(qcd_df.index, year, month_idx + 1) and is_dfs_index_has_same_range(min_df.index, qcd_df.index):
                    print(stn_id, year, month_idx+1)
                
            # 한쪽만 있는 월. 크게 의미가 없다.
            else:
                print(stn_id, year, month_idx+1)

2021:  11%|█         | 82/761 [01:13<10:09,  1.11it/s]


KeyboardInterrupt: 

TODO - 시각화(full range 비율, 기상요소 사용비율, 결측치 비율 등)